<a href="https://colab.research.google.com/github/DARSHANH-analytics/ANN/blob/master/Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install pyspark
from pyspark import SparkContext,SQLContext, SparkConf,StorageLevel
import numpy as np
from decimal import Decimal
import matplotlib.pyplot as plt;plt.rcdefaults()
from __future__ import division

In [2]:
SparkConf = SparkConf().setAppName("ODI").setMaster("local[3]")
# here local : Run Spark locally with one worker thread (i.e. no parallelism at all).
# local[K] : Run Spark locally with K worker threads (ideally, set this to the number of cores on your machine).
sc = SparkContext(conf=SparkConf)

In [20]:
Player_odis = sc.textFile("/content/drive/MyDrive/Colab Notebooks/ODI_data.csv",4,use_unicode=True).repartition(6)

In [21]:
Player_odis.take(5)

['0,GW Flower (ZIM),19922010,221,214,18,6571,142*,33.52,9723,67.58,6,40,18',
 '1,Ijaz Ahmed (PAK),19862000,250,232,29,6564,139*,32.33,8174,80.3,10,37,14',
 '2,AR Border (AUS),19791994,273,252,39,6524,127*,30.62,9134,71.42,3,39,11',
 '3,Shakib Al Hasan (BDESH),20062019,206,194,27,6323,134*,37.86,7641,82.75,9,47,10',
 '4,BRM Taylor (ZIM),20042019,193,192,15,6293,145*,35.55,8256,76.22,10,38,15']

In [22]:
columns = 'SI_NO,Player,Span,Mat,Inns,NO,Runs,HS,Ave,BF,SR,No_of_100s,No_of_50s,No_of_0s'.split(',')
tuple(columns)

('SI_NO',
 'Player',
 'Span',
 'Mat',
 'Inns',
 'NO',
 'Runs',
 'HS',
 'Ave',
 'BF',
 'SR',
 'No_of_100s',
 'No_of_50s',
 'No_of_0s')

In [24]:
from collections import namedtuple
Player = namedtuple('Player',tuple(columns))
def parserec(line):
  fields = line.split(',')
  for i in range(0,len(fields)):
    if fields[i] is None or fields[i] == '':
      fields[i] = 0
  return Player(int(fields[0]),fields[1],int(fields[2]),int(fields[3]),int(fields[4]),int(fields[5]),int(fields[6]),fields[7],float(fields[8]),\
                int(fields[9]),float(fields[10]),int(fields[11]),int(fields[12]),int(fields[13]))
Players = Player_odis.map(lambda rec: parserec(rec))

In [25]:
Players.take(10)
# READING first 10 records

[Player(SI_NO=0, Player='GW Flower (ZIM)', Span=19922010, Mat=221, Inns=214, NO=18, Runs=6571, HS='142*', Ave=33.52, BF=9723, SR=67.58, No_of_100s=6, No_of_50s=40, No_of_0s=18),
 Player(SI_NO=1, Player='Ijaz Ahmed (PAK)', Span=19862000, Mat=250, Inns=232, NO=29, Runs=6564, HS='139*', Ave=32.33, BF=8174, SR=80.3, No_of_100s=10, No_of_50s=37, No_of_0s=14),
 Player(SI_NO=2, Player='AR Border (AUS)', Span=19791994, Mat=273, Inns=252, NO=39, Runs=6524, HS='127*', Ave=30.62, BF=9134, SR=71.42, No_of_100s=3, No_of_50s=39, No_of_0s=11),
 Player(SI_NO=3, Player='Shakib Al Hasan (BDESH)', Span=20062019, Mat=206, Inns=194, NO=27, Runs=6323, HS='134*', Ave=37.86, BF=7641, SR=82.75, No_of_100s=9, No_of_50s=47, No_of_0s=10),
 Player(SI_NO=4, Player='BRM Taylor (ZIM)', Span=20042019, Mat=193, Inns=192, NO=15, Runs=6293, HS='145*', Ave=35.55, BF=8256, SR=76.22, No_of_100s=10, No_of_50s=38, No_of_0s=15),
 Player(SI_NO=5, Player='RB Richardson (WI)', Span=19831996, Mat=224, Inns=217, NO=30, Runs=6248, H

In [30]:
Players_Mat200 = Players.filter(lambda rec:rec.SI_NO < 10)
Players_100s_SR = Players_Mat200.filter(lambda rec:rec.No_of_100s>=10 and rec.SR>80)
Players_100s_SR.collect()
# Applying filters transformation and specifying action

[Player(SI_NO=1, Player='Ijaz Ahmed (PAK)', Span=19862000, Mat=250, Inns=232, NO=29, Runs=6564, HS='139*', Ave=32.33, BF=8174, SR=80.3, No_of_100s=10, No_of_50s=37, No_of_0s=14),
 Player(SI_NO=7, Player='KS Williamson (NZ)', Span=20102019, Mat=149, Inns=142, NO=14, Runs=6132, HS='148', Ave=47.9, BF=7494, SR=81.82, No_of_100s=13, No_of_50s=39, No_of_0s=5),
 Player(SI_NO=0, Player='SR Tendulkar (INDIA)', Span=19892012, Mat=463, Inns=452, NO=41, Runs=18426, HS='200*', Ave=44.83, BF=21367, SR=86.23, No_of_100s=49, No_of_50s=96, No_of_0s=20),
 Player(SI_NO=2, Player='RT Ponting (AUS/ICC)', Span=19952012, Mat=375, Inns=365, NO=39, Runs=13704, HS='164', Ave=42.03, BF=17046, SR=80.39, No_of_100s=30, No_of_50s=82, No_of_0s=20),
 Player(SI_NO=3, Player='ST Jayasuriya (Asia/SL)', Span=19892011, Mat=445, Inns=433, NO=18, Runs=13430, HS='189', Ave=32.36, BF=14725, SR=91.2, No_of_100s=28, No_of_50s=68, No_of_0s=34),
 Player(SI_NO=6, Player='V Kohli (INDIA)', Span=20082019, Mat=242, Inns=233, NO=39, 